<a href="https://colab.research.google.com/github/Emmanuel-K07/chatterbox/blob/master/stable/526_mix_comfyui_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ComfyUI Installation for Google Colab - Version stable et moderne
# Évite les conflits de dépendances tout en gardant l'écosystème Colab intact

import subprocess
import sys
import os
from pathlib import Path

def run_cmd(cmd, check=True, capture=False):
    """Execute command with proper error handling"""
    result = subprocess.run(cmd, shell=True, capture_output=capture, text=True)
    if check and result.returncode != 0:
        print(f"❌ Erreur: {cmd}")
        if capture:
            print(f"STDOUT: {result.stdout}")
            print(f"STDERR: {result.stderr}")
        return False
    return result.stdout if capture else True

print("🚀 Installation ComfyUI pour Google Colab")
print("=" * 50)

# 1. Préparation environnement
print("📦 Préparation de l'environnement...")
os.chdir('/content')

# Créer un environnement virtuel isolé pour ComfyUI
if not os.path.exists('/content/comfyui_env'):
    run_cmd('python -m venv comfyui_env')

# 2. Installation PyTorch compatible
print("🔥 Installation PyTorch + CUDA 11.8...")
pytorch_cmd = """
source /content/comfyui_env/bin/activate && \
pip install --upgrade pip wheel setuptools && \
pip install torch==2.1.0 torchvision==0.16.0 torchaudio==2.1.0 --index-url https://download.pytorch.org/whl/cu118
"""
run_cmd(pytorch_cmd)

# 3. Installation xformers compatible avec PyTorch 2.1
print("⚡ Installation xformers optimisé...")
xformers_cmd = """
source /content/comfyui_env/bin/activate && \
pip install xformers==0.0.22.post4 --index-url https://download.pytorch.org/whl/cu118
"""
run_cmd(xformers_cmd)

# 4. Clone ComfyUI + Manager
print("📥 Téléchargement ComfyUI...")
if not os.path.exists('/content/ComfyUI'):
    run_cmd('git clone https://github.com/comfyanonymous/ComfyUI.git')

os.chdir('/content/ComfyUI')

# Installation ComfyUI Manager pour faciliter l'ajout de nodes
if not os.path.exists('/content/ComfyUI/custom_nodes/ComfyUI-Manager'):
    run_cmd('git clone https://github.com/ltdrdata/ComfyUI-Manager.git /content/ComfyUI/custom_nodes/ComfyUI-Manager')

# 5. Installation dépendances ComfyUI dans l'environnement isolé
print("📚 Installation dépendances ComfyUI...")
deps_cmd = """
source /content/comfyui_env/bin/activate && \
pip install -r requirements.txt && \
pip install opencv-python-headless pillow numpy scipy scikit-image transformers accelerate
"""
run_cmd(deps_cmd)

# 6. Installation nodes essentiels pour votre workflow
print("🔌 Installation des custom nodes essentiels...")
essential_nodes = [
    'https://github.com/Fannovel16/comfyui_controlnet_aux.git',  # ControlNet preprocessors
    'https://github.com/cubiq/ComfyUI_IPAdapter_plus.git',       # IPAdapter avancé
    'https://github.com/ArtVentureX/comfyui-animatediff.git',   # AnimateDiff
    'https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite.git', # Video processing
    'https://github.com/WASasquatch/was-node-suite-comfyui.git'    # Utilities
]

for node_url in essential_nodes:
    node_name = node_url.split('/')[-1].replace('.git', '')
    if not os.path.exists(f'/content/ComfyUI/custom_nodes/{node_name}'):
        run_cmd(f'git clone {node_url} /content/ComfyUI/custom_nodes/{node_name}')

# 7. Installation dépendances des custom nodes
print("🔧 Installation dépendances des custom nodes...")
for node_dir in Path('/content/ComfyUI/custom_nodes').iterdir():
    if node_dir.is_dir() and (node_dir / 'requirements.txt').exists():
        req_cmd = f"""
        source /content/comfyui_env/bin/activate && \
        pip install -r {node_dir}/requirements.txt
        """
        run_cmd(req_cmd, check=False)  # Don't fail if some requirements conflict

# 8. Téléchargement modèles de base
print("🎨 Téléchargement des modèles essentiels...")
models_dir = Path('/content/ComfyUI/models')

# Créer les dossiers de modèles
for subdir in ['checkpoints', 'controlnet', 'ipadapter', 'vae', 'clip_vision']:
    (models_dir / subdir).mkdir(parents=True, exist_ok=True)

# Télécharger SDXL base (essentiel pour votre workflow)
sdxl_path = models_dir / 'checkpoints' / 'sd_xl_base_1.0.safetensors'
if not sdxl_path.exists():
    print("📥 Téléchargement SDXL Base (3.5GB)...")
    run_cmd(f'wget -O {sdxl_path} https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors')

# ControlNet Depth pour SDXL
controlnet_path = models_dir / 'controlnet' / 'diffusers_xl_depth_full.safetensors'
if not controlnet_path.exists():
    print("📥 Téléchargement ControlNet Depth SDXL...")
    run_cmd(f'wget -O {controlnet_path} https://huggingface.co/diffusers/controlnet-depth-sdxl-1.0/resolve/main/diffusion_pytorch_model.safetensors')

# IPAdapter pour SDXL
ipadapter_path = models_dir / 'ipadapter' / 'ip_adapter_xl.safetensors'
if not ipadapter_path.exists():
    print("📥 Téléchargement IPAdapter SDXL...")
    run_cmd(f'wget -O {ipadapter_path} https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl.safetensors')

# CLIP Vision pour IPAdapter
clip_path = models_dir / 'clip_vision' / 'CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors'
if not clip_path.exists():
    print("📥 Téléchargement CLIP Vision...")
    run_cmd(f'wget -O {clip_path} https://huggingface.co/h94/IP-Adapter/resolve/main/models/image_encoder/model.safetensors')

# 9. Configuration Cloudflare tunnel
print("🌐 Configuration du tunnel Cloudflare...")
if not os.path.exists('/content/cloudflared'):
    run_cmd('wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared')
    run_cmd('chmod +x /content/cloudflared')

# 10. Script de lancement
launch_script = '''#!/bin/bash
source /content/comfyui_env/bin/activate
cd /content/ComfyUI

# Lancer Cloudflare tunnel en arrière-plan
/content/cloudflared tunnel --url http://127.0.0.1:8188 &
sleep 5

# Récupérer l'URL du tunnel
TUNNEL_URL=$(curl -s http://127.0.0.1:8080/metrics | grep -o 'https://[^"]*trycloudflare.com')
echo "🌐 ComfyUI accessible sur: $TUNNEL_URL"

# Lancer ComfyUI
python main.py --listen 127.0.0.1 --port 8188
'''

with open('/content/launch_comfyui.sh', 'w') as f:
    f.write(launch_script)
run_cmd('chmod +x /content/launch_comfyui.sh')

print("✅ Installation terminée!")
print("=" * 50)
print("🚀 Pour lancer ComfyUI:")
print("   !bash /content/launch_comfyui.sh")
print()
print("📁 Dossiers importants:")
print(f"   • ComfyUI: /content/ComfyUI")
print(f"   • Modèles: /content/ComfyUI/models")
print(f"   • Custom nodes: /content/ComfyUI/custom_nodes")
print()
print("💡 Conseils:")
print("   • Uploadez votre meuble PNG dans /content/ComfyUI/input/")
print("   • Le workflow JSON que j'ai créé est compatible")
print("   • Utilisez ComfyUI Manager pour installer des nodes supplémentaires")

In [ ]:
pip install --force-reinstall torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio --index-url https://download.pytorch.org/whl/cu118
